In [ ]:
!gsutil cp -r gs://dataset_eeg_cafe2022 data

In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets